# Импорт библиотек

In [1]:
from selenium import webdriver
import time
from datetime import datetime
import re
import requests
import random
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Запуск WB

In [2]:
##################################################################################################################
# подключаем опции для объявления юзер-агента
options = webdriver.ChromeOptions()
#options.add_argument(f"user-agent={random.choice(user_agent_list)}")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/"+
                     "537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36")
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")

# Запуск веб-драйвера
#service = Service(executable_path=r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\Парсер яндекса\chromedriver.exe")
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 60)

url = "https://www.wildberries.ru/"
driver.get(url=url)
time.sleep(3)

# Установка запроса

In [3]:
search = 'Электроника'

# Ввод запроса

In [4]:
# Вводим строку поиска
input_place = driver.find_element(By.CSS_SELECTOR, "#searchInput".format('Найти на Wildberries'))
input_place.clear()
input_place.send_keys(search)
input_place.send_keys(Keys.ENTER)

time.sleep(5)

# Создание датафрейма для собранных данных

In [5]:
data = pd.DataFrame(columns = ['ИНН', 'Ссылка на карточку', 'Название', 'Фокусная отрасль(запрос)', 'Количество товаров продано', 'Официальное название'])
data

,ИНН,Ссылка на карточку,Название,Фокусная отрасль(запрос),Количество товаров продано,Официальное название


# Создание списка с уже сохраненными селлерами из ранее сохраненных экселек

In [6]:
df1 = pd.read_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Первая партия\объединенный.xlsx")
df2 = pd.read_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Вторая партия\Фэшен.xlsx")
df3 = pd.read_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Третья партия\Фэшен1.xlsx")
df4 = pd.read_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Третья партия\Фэшен2.xlsx")
df5 = pd.read_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Четвертая партия\Фэшен.женская одежда.платья6.xlsx")
df6 = pd.read_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Четвертая партия\Фэшен.женская одежда.блузки3.xlsx")
df7 = pd.read_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Четвертая партия\Фэшен.женская одежда.юбки2.xlsx")
df8 = pd.read_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Четвертая партия\Фэшен.женская одежда.брюки2.xlsx")
df9 = pd.read_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Четвертая партия\Фэшен.женская одежда.верхняя одежда2.xlsx")
df10 = pd.read_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Четвертая партия\Фэшен.мужская одежда.рубашки2.xlsx")
df11 = pd.read_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Четвертая партия\Смартфоны10.xlsx")
df12 = pd.read_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Пятая партия\Наушники15.xlsx")
df13 = pd.read_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Шестая партия\Обувь9.xlsx")
df14 = pd.read_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Седьмая партия\Умные часы9.xlsx")
df15 = pd.read_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Седьмая партия\Уходовая косметика12.xlsx")
df16 = pd.read_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Седьмая партия\Декоративная косметика5.xlsx")
df17 = pd.read_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Седьмая партия\Парфюмерия10.xlsx")
df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15, df16, df17], axis = 0)
old_old_names = list(df['Название'])
old_old_names
len(old_old_names)

2771

# Сам парсинг

Лучше перевести Chrome в половину экрана, иначе может не работать из-за другого отображения страницы

In [7]:
old_names = list(data['Название'])#OLD_NAMES - УЖЕ СОБРАННЫЕ ИМЕНА ИЗ ТЕКУЩЕГО ДАТАФРЕЙМА (НЕ ИЗ ТЕХ, КОТОРЫЕ БЕРЕМ ИЗ УЖЕ СОХРАНЕННЫХ ЭКСЕЛЕК) 
driver.switch_to.window(driver.window_handles[0])# ПЕРЕКЛЮЧЕНИЕ НА НУЛЕВУЮ ВКЛАДКУ (С КАТАЛОГОМ)
#СОЗДАНИЕ СПИСКА ВСЕХ ССЫЛОК НА КАРТОЧКИ ПРОДУКТОВ 
html_source = driver.page_source#КОД СТРАНИЦЫ ДЛЯ BS
soup = BeautifulSoup(html_source, 'html.parser')
links = soup.find_all('a', class_='product-card__link')#ПОИСК ВСЕХ ССЫЛОК НА КАРТОЧКИ ПРОДУКТОВ
href_values = [link['href'] if link else None for link in links]#ВЫДЕЛЕНИЕ ССЫЛОК
for i in range(len(href_values)):
    print(i)#НОМЕР ТОВАРНОЙ КАРТОЧКИ
    old_names = list(data['Название'])#OLD_NAMES - УЖЕ СОБРАННЫЕ ИМЕНА ИЗ ТЕКУЩЕГО ДАТАФРЕЙМА (НЕ ИЗ ТЕХ, КОТОРЫЕ БЕРЕМ ИЗ УЖЕ СОХРАНЕННЫХ ЭКСЕЛЕК)
    try:
        driver.execute_script("window.open(arguments[0], '_blank');", href_values[i])#ОТКРЫТЬ В НОВОЙ ВКЛАДКЕ I-УЮ ССЫЛКУ
        driver.switch_to.window(driver.window_handles[2])#ПЕРЕЙТИ НА ЭТУ ВКЛАДКУ
        time.sleep(1)

        try:
            element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "product-page")))#ДОЖИДАЕМСЯ ПОКА НЕ ЗАГРУЗИТСЯ СТРАНИЦА ТОВАРА
            html_source = driver.page_source
            soup = BeautifulSoup(html_source, 'html.parser')
            seller_name_div = soup.find('div', class_='seller-and-brand__item-name')#НАХОДИМ ИМЯ ПРОДАВЦА В HTML КОДЕ СТРАНИЦЫ
            seller_name = seller_name_div.get_text(strip=True) if seller_name_div else None
            if seller_name == None:# ЕСЛИ НЕ НАЙДЕНО ИМЯ ПРОДАВЦА, КАЖЕТСЯ ТАКОГО НИКОГДА НЕ БЫЛО
                print("seller not found")
            if seller_name in old_names or seller_name == None or seller_name in old_old_names:#ЕСЛИ ЭТОТ ПРОДАВЕЦ УЖЕ БЫЛ СОХРАНЕН
                print("already seen this seller")
                continue
            seller_link = soup.find('a', class_='seller-and-brand__link')# ПОИСК ОБЪЕКТА ССЫЛКИ НА ПРОДАВЦА 
            seller_link = seller_link['href'] if seller_link else None#ВЫДЕЛЕНИЕ ТЕКСТА ССЫЛКИ
            time.sleep(0.5)
            try:
                driver.execute_script("window.open(arguments[0], '_blank');", seller_link)# ОТКРЫТИЕ ПРОДАВЦА В НОВОЙ ВКЛАДКЕ
                driver.switch_to.window(driver.window_handles[3])# ПЕРЕКЛЮЧЕНИЕ НА ЭТУ ВКЛАДКУ
                time.sleep(1)
                
                try:
                    element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "seller-details")))# ОЖИДАНИЕ ЗАГРУЗКИ СТРАНИЦЫ ПРОДАВЦА
                    html_source = driver.page_source
                    soup = BeautifulSoup(html_source,"html.parser")
                    
                    #КОЛИЧЕСТВО ТОВАРОВ ПРОДАНО НА WB
                    #_____________________________________________________
                    
                    sold_items_element = soup.find('p', class_='seller-details__parameter-value--delivered')
                    if sold_items_element:
                        sold_items = sold_items_element.text.strip() 
                        print(f'Число проданных товаров: {sold_items}')
                    else:
                       print('Число проданных товаров не найдено')
                       sold_items = '?'
                except:
                   print("didn't get amount sold")
                   sold_items = '?'
                    
                #________________________________________________________
                try:
                    info = driver.find_element(By.CLASS_NAME, 'seller-details__tip-info')# КЛИК НА КНОПКУ INFO ДЛЯ ПОЛУЧЕНИЯ ИНН И ОФФИЦИАЛЬНОГО НАЗВАНИЯ
                    info.click()
                    print("info clicked")
                    time.sleep(1)
                    #ПОИСК ИНН:
                    #_____________________________________________________________
                    try:
                        html_source = driver.page_source
                        soup = BeautifulSoup(html_source,"html.parser")
                        inn_element = soup.find('p', text=lambda x: x and 'ИНН' in x)
                        if inn_element:
                            inn_number = inn_element.text.split(': ')[1]
                            print(f'ИНН: {inn_number}')
                        else:
                            
                            inn_number = '?'        
                    except:
                        inn_number = '?'

                    #____________________________________________________________

                    #ПОИСК ОФИЦИАЛЬНОГО НАЗВАНИЯ
                    #____________________________________________________________
                    try:
                        html_code = driver.page_source
                        soup = BeautifulSoup(html_code, 'html.parser')
                        entrepreneur_info = soup.find('div', class_='tooltip__content content--L91D5').find('p')
                        entrepreneur_name = entrepreneur_info.get_text(strip=True) if entrepreneur_info else None
                        official_name = str(entrepreneur_info)[3:-4]
                    except:
                        official_name = '?'

                    #____________________________________________________________
                except:
                    print("info not clicked")
                finally:
                    #СБОР ССЫЛКИ НА КАРТОЧКУ
                    card_link = driver.current_url
                    #СТРОКА С СОБРАННЫМИ ЗНГАЧЕНИЯМИ
                    data_string = [inn_number, card_link, seller_name, search, sold_items, official_name]
                    print(data_string)
                    #ДОБАВЛЕНИЕ В ДАТАФРЕЙМ
                    data = pd.concat([data, pd.DataFrame([data_string], columns = data.columns)], ignore_index=True)
                    #ЗАКРЫТИЕ ВКЛАДКИ ПРОДАВЦА
                    driver.close()
                    #ПЕРЕКЛЮЧЕНИЕ НА ПРЕДЫДУЩУЮ ВКЛАДКУ
                    driver.switch_to.window(driver.window_handles[2])
            except:
                print('seller not opened')
        except Exception as Ex:
            print('something went wrong with reading html')
            print(f'Ошибка: {Ex}')
        finally:
            #ЗАКРЫТИЕ ВКЛАДКИ ТОВАРА И ПЕРЕКЛЮЧЕНИЕ НА ВКЛАДКУ КАТАЛОГА
            time.sleep(0.2)
            driver.close()
            time.sleep(0.1)
            driver.switch_to.window(driver.window_handles[0])
            print("closed")
    except:
        print("card not opened")
data

0
card not opened
1
Число проданных товаров: 242 623
info clicked


C:\Users\ermakov.dst\AppData\Local\Temp\ipykernel_1124\2164984988.py:67: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  inn_element = soup.find('p', text=lambda x: x and 'ИНН' in x)


ИНН: 7733408065
['7733408065', 'https://www.wildberries.ru/seller/1231110', 'Envitec', 'Электроника', '242 623', 'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ГУАНСИ"']
closed
2
Число проданных товаров: 57 833
info clicked
ИНН: 564802905405
['564802905405', 'https://www.wildberries.ru/seller/1401034', 'Motor Mind', 'Электроника', '57 833', 'Индивидуальный предприниматель Григорьев Иван Александрович']
seller not opened
card not opened
3
card not opened
4
card not opened
5
card not opened
6
card not opened
7
card not opened
8
card not opened
9
card not opened
10
card not opened
11
card not opened
12
card not opened
13
card not opened
14
card not opened
15
card not opened
16
card not opened
17
card not opened
18
card not opened
19
card not opened
20
card not opened
21
card not opened
22
card not opened
23
card not opened
24
card not opened
25
card not opened
26
card not opened
27
card not opened
28
card not opened
29
card not opened


,ИНН,Ссылка на карточку,Название,Фокусная отрасль(запрос),Количество товаров продано,Официальное название
0,7733408065,https://www.wildberries.ru/seller/1231110,Envitec,Электроника,242 623,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ГУАНСИ"""
1,564802905405,https://www.wildberries.ru/seller/1401034,Motor Mind,Электроника,57 833,Индивидуальный предприниматель Григорьев Иван ...


# очистка дубликатов

In [168]:
data = data.drop_duplicates(subset = ['ИНН'])
data

,ИНН,Ссылка на карточку,Название,Фокусная отрасль(запрос),Количество товаров продано,Официальное название
0,0800029136,https://www.wildberries.ru/seller/250001745,СОФИ БОНТЕ,Парфюмерия,10 657,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""СОФИ..."
1,770207140329,https://www.wildberries.ru/seller/912777,Ramzi,Парфюмерия,24 348,Индивидуальный предприниматель Мирзосанова Рам...
2,695012685320,https://www.wildberries.ru/seller/1190271,Магия ароматов,Парфюмерия,153 683,Индивидуальный предприниматель Алдакулова Бега...
3,7727491227,https://www.wildberries.ru/seller/726190,STELA MISS,Парфюмерия,58 891,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""СТЕЛ..."
4,644909870297,https://www.wildberries.ru/seller/458185,M&R Perfume,Парфюмерия,624 460,Индивидуальный предприниматель Мамедов Рафик А...
...,...,...,...,...,...,...
239,502758727773,https://www.wildberries.ru/seller/213915,Aroma shop,Парфюмерия,17 872,Индивидуальный предприниматель Алиев Рустам Су...
240,201677441448,https://www.wildberries.ru/seller/280155,SOAYPPERFUME,Парфюмерия,125 440,Индивидуальный предприниматель Коруев Соайп Са...
241,667008026801,https://www.wildberries.ru/seller/96419,АПРЕЛЬ96,Парфюмерия,24 944,ИП Петрова Инна Александровна
242,9721215506,https://www.wildberries.ru/seller/1406535,Branded perfumes,Парфюмерия,61 077,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""АМАНАТ"""


# Сохранение в Excel

In [173]:
data.to_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Седьмая партия\Парфюмерия10.xlsx")

# Сохранение всего собранного

In [3]:
df.to_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Седьмая партия\промежуточный итог18.06.xlsx")
df

,ИНН,Ссылка на карточку,Название,Фокусная отрасль(запрос),Количество товаров продано,Unnamed: 0,Официальное название
0,561507269404,https://www.wildberries.ru/seller/455568,MaryashA,Фэшен,149 268,NaN,NaN
1,665821766737,https://www.wildberries.ru/seller/4234077,B-Store,Фэшен,70 652,NaN,NaN
2,290209522869,https://www.wildberries.ru/seller/3945406,Brenda Fashion,Фэшен,8 964,NaN,NaN
3,22301565032,https://www.wildberries.ru/seller/272815,SG fashion & accessory,Фэшен,361 529,NaN,NaN
4,773001860640,https://www.wildberries.ru/seller/78504,B.B.Fashion,Фэшен,33 674,NaN,NaN
...,...,...,...,...,...,...,...
254,070711860657,https://www.wildberries.ru/seller/1402440,Adam,Парфюмерия,23 834,254.0,Индивидуальный предприниматель Баркал Гада
255,165064455557,https://www.wildberries.ru/seller/4248052,Парфюмные хиты,Парфюмерия,4 860,255.0,Индивидуальный предприниматель Тилеков Чынтемир
256,622809640920,https://www.wildberries.ru/seller/624327,Арома Мэджик,Парфюмерия,77 844,256.0,Индивидуальный предприниматель Богословская Лю...
257,350705516274,https://www.wildberries.ru/seller/4025176,Perfume Dreams,Парфюмерия,41,257.0,Индивидуальный предприниматель Атакишиева Айгю...
